# pybela Tutorial 7: Sparse timestamping
In the potentiometer example used in the previous tutorials, the values for `pot1` and `pot2` are assigned at every audio frame. Let's take a look again at the `render()` loop (the Bela code for this example can be found in  (in `bela-code/potentiometers/render.cpp`).

```cpp
void render(BelaContext *context, void *userData)
{
	for(unsigned int n = 0; n < context->audioFrames; n++) {
		if(gAudioFramesPerAnalogFrame && !(n % gAudioFramesPerAnalogFrame)) {
			
			uint64_t frames = context->audioFramesElapsed/gAudioFramesPerAnalogFrame + n/gAudioFramesPerAnalogFrame;
			Bela_getDefaultWatcherManager()->tick(frames); // watcher timestamps
			
			pot1 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot1Ch);
			pot2 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot2Ch);
			
		}
	}
}
```


The Watched clock is also "ticked" at every analog frame, so that the timestamps in the data correspond to the audio frames in the Bela code. The data buffers we received from Bela in the Streamer and the Logger had this form: `{"ref_timestamp": 92381, "data":[0.34, 0.45, ...]}`. Each data point is registered in the buffer every time we assign a value to `pot1` and `pot2` in the Bela code. The `ref_timestamp` corresponds to the timestamp of the first sample in the `data` array, in this case `0.34`. Since in the Bela code, we assign `pot1` and `pot2` at every audio frame, we can infer the timestamps of each value in the data array by incrementing `ref_timestamp` by 1 for each sample. 

This is an efficient way of storing data since instead of storing the timestamp of every item in the data array, we only store the timestamp of the first item. We call this *dense* timestamping. However, for many applications, we might not assign a value to a variable every frame, we might do it more than once per frame, once every few frames, or we might want to do it at irregular intervals. In these cases, we need to store the timestamp of every item in the data array. We call this *sparse* timestamping.

In this tutorial we take a look at *sparse* timestamping. The complete documentation for the pybela library can be found in [https://belaplatform.github.io/pybela/](https://belaplatform.github.io/pybela/).

First, transfer the Bela code we will use in this tutorial to Bela:


In [ ]:
!rsync -rvL ../bela-code/timestamping root@bela.local:Bela/projects

Then you can compile and run the project using either the IDE or by running the following command in the Terminal:
```bash
ssh root@bela.local "make -C Bela stop Bela PROJECT=potentiometers run" 
```
(Running this on a jupyter notebook will block the cell until the program is stopped on Bela.)

As in the previous tutorials, we will use two potentiometers connected to Bela analog inputs 0 and 1. Check the  `1_Streamer.ipnyb` tutorial notebook for instructions on how to set up the circuit. 

### Bela C++ code


First, let's take a look at the Bela code. First, we have added `WatcherManager::kTimestampSample` to the declaration of `pot2`. This informs the Bela Watcher that `pot2` will be watched sparsely, that is, that the watcher will store a timestamp for every value assigned to `pot2`:

```cpp
Watcher<float> pot1("pot1");
Watcher<float> pot2("pot2",  WatcherManager::kTimestampSample);
```

Now let's take a look at `render()`:

```cpp
void render(BelaContext *context, void *userData)
{
	for(unsigned int n = 0; n < context->audioFrames; n++) {
		if(gAudioFramesPerAnalogFrame && !(n % gAudioFramesPerAnalogFrame)) {
			
			uint64_t frames = context->audioFramesElapsed/gAudioFramesPerAnalogFrame + n/gAudioFramesPerAnalogFrame;
			Bela_getDefaultWatcherManager()->tick(frames); // watcher timestamps
			
			pot1 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot1Ch);

			if (frames % 12==0){
				pot2 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot2Ch);
			}
		}
	}
}
```

We are "ticking" the Bela Watcher once per analog frame, so that the timestamps in the data correspond to the analog frames in the Bela code. We are assigning a value to `pot1` at every analog frame, as in the previous examples, but we are now only assigning a value to `pot2` every 12 frames. 

### Dealing with sparse timestamps in Python

Let's now take a look at the data we receive from Bela. We will use the Streamer. Run the cells below to declare and connect the Streamer to Bela:

In [ ]:
import asyncio
import pandas as pd
from pybela import Streamer

In [ ]:
streamer = Streamer()
streamer.connect()

We can call `.list()`  to take a look at the variables available to be streamed, their types and timestamp mode:

In [ ]:
streamer.list()

`timestampMode` indicates if the timestamping is *sparse* (1) or *dense* (0). Now let's stream the data from Bela. We will stream `pot1` and `pot2`:

In [ ]:
streamer.start_streaming(variables=["pot1", "pot2"], saving_enabled=False)
await asyncio.sleep(2)
streamer.stop_streaming()

Now let's take a look at the streamed buffers for "pot2". Each buffer has the form `{"ref_timestamp": 912831, "data":[0.23, 0.24, ...], "rel_timestamps":[ 0, 12, ...]}`. `ref_timestamp` corresponds, as in the dense case, to the timestamp of the first data point in the `data` array. `rel_timestamps` is an array of timestamps relative to `ref_timestamp`. In this case, since we are assigning a value to `pot2` every 12 frames, the timestamps in `rel_timestamps` are `[0, 12, 24, 36, etc.]`.

In [ ]:
streamer.streaming_buffers_queue["pot2"]

You can now calculate the absolute timestamps of each data point by adding the values in `rel_timestamps` to `ref_timestamp`:

In [ ]:
[streamer.streaming_buffers_queue["pot2"][0]["ref_timestamp"]]*len(streamer.streaming_buffers_queue["pot2"][0]["rel_timestamps"]) + streamer.streaming_buffers_queue["pot2"][0]["rel_timestamps"]

In [ ]:
pot2_data = {"timestamps":[], "data":[]}

for _buffer in streamer.streaming_buffers_queue["pot2"]:
    pot2_data["timestamps"].extend([_buffer["ref_timestamp"] + i for i in _buffer["rel_timestamps"]])
    pot2_data["data"].extend(_buffer["data"])

Note that the timestamps are spaced by 12, as expected:

In [ ]:
df = pd.DataFrame(pot2_data)
df.head()